# GSMLS

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from IPython.core.display import display, HTML
from concurrent.futures import ThreadPoolExecutor

pd.set_option('display.max_rows', None)

def preview_gsmls_df(mydf, save=False):
    columns = [
        'address', 'city', 'county', 'bedrooms', 'baths_full', 'baths_part',
        'sqft', 'price', 'tax', 'style', 'rooms', 'park_and_ride_name',
        'park_and_ride_duration_text', 'park_and_ride_type', 'park_and_ride_distance',
        'park_and_ride_bus_name', 'park_and_ride_bus_distance',
        'barnabas_duration_text', 'nyc_duration_text',]
    if save:
        mydf[columns].to_csv('gsmls.csv')
    display(HTML(
        """<style>.dataframe td {white-space: nowrap;}</style>""" +
        mydf[columns].to_html(
            formatters={
                '__index__':
                    lambda id: f'<a target="_blank" href="https://gsmls.herokuapp.com/properties/{id}">{id}</a>'},
                        escape=False)))

**Define global parameters**

In [3]:
counties = ['Passaic', 'Hudson','Essex', 'Middlesex', 'Morris', 'Bergen', 'Union', 'Sussex',]

ONE_HOUR = 3600
TEN_MINUTES = ONE_HOUR/6
HALF_HOUR = ONE_HOUR / 2
ONE_HOUR_TEN_MINUTES = ONE_HOUR + TEN_MINUTES
ONE_HOUR_THIRTY_MINUTES = ONE_HOUR + HALF_HOUR
ONE_HOUR_FORTY_MINUTES = ONE_HOUR + HALF_HOUR + TEN_MINUTES

blacklist_cities = [
    'Paterson', 'East Orange', 'City of Orange', 'Belleville', 'Elizabeth',
    'Jefferson', 'Passaic', 'Newark', 'East Newark', 'Linden', 'Nutley',
    'Garfield', 'Hillside', 'Little Ferry']

**Download listings from [GSMLS](https://www.gsmls.com/)**

In [4]:
from gsmls import get_listings

listings = []
for county in tqdm(counties):

    listings += get_listings(county,
                           min_list_price=350000,
                           max_list_price=400000,
                           min_bedrooms=3,
                           min_bathrooms=2)

    listings += get_listings(county,
                           min_list_price=400000,
                           max_list_price=515000,
                           min_bedrooms=3,
                           min_bathrooms=2)

print(f"Downloaded {len(listings)} listings from GSMLS.")
gsmls_listings = [x for x in listings if x['address']]
print(f"Filtered down to {len(listings)} listings due to null address.")

HBox(children=(IntProgress(value=0, max=8), HTML(value='')))


Downloaded 1106 listings from GSMLS.
Filtered down to 1106 listings due to null address.


**Geocode each address using Google Maps API**

In [5]:
from geolocate import add_geocode_to_listing
for listing in tqdm(listings):
    if not listing.get('address'): continue
    add_geocode_to_listing(listing)

HBox(children=(IntProgress(value=0, max=1106), HTML(value='')))

could not find county for 3426835 46 Veranda Ave, North Caldwell, NJ, USA [{'long_name': '46', 'short_name': '46', 'types': ['street_number']}, {'long_name': 'Veranda Avenue', 'short_name': 'Veranda Ave', 'types': ['route']}, {'long_name': 'North Caldwell', 'short_name': 'North Caldwell', 'types': ['locality', 'political']}, {'long_name': 'New Jersey', 'short_name': 'NJ', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}]
could not geocode: 4 Shadtree Lane Mount Arlington Boro, NJ or 4 Shadtree Lane Mount Arlington, NJ
could not geocode: 2 Shadetree Lane Mount Arlington Boro, NJ or 2 Shadetree Lane Mount Arlington, NJ
could not geocode: 15 Shadetree Ln Mount Arlington Boro, NJ or 15 Shadetree Ln Mount Arlington, NJ
could not geocode: 1 Harbor Front Plz A1 Elizabeth City, NJ or 1 Harbor Front Plz A1 Elizabeth, NJ
could not geocode: 1 Harbor Front Ter D1 Elizabeth City, NJ or 1 Harbor Front Ter D1

**Find closest NJ Transit Park and Ride to each address**

In [6]:
from commute import add_closest_park_and_ride_to_listing

for i in tqdm(range(len(listings))):
    listing = listings[i]
    add_closest_park_and_ride_to_listing(listing)

HBox(children=(IntProgress(value=0, max=1106), HTML(value='')))

**Get commute time for each address through both park and ride and walking**

In [7]:
from commute import add_commute_to_listing

for i in tqdm(range(len(listings))):
    listing = listings[i]
    add_commute_to_listing(listing)

print(sum(1 for listing in listings if listing.get('park_and_ride_duration')),
      "out of", len(listings), "have park and ride commute times.")

print(sum(1 for listing in listings if listing.get('nyc_duration')),
  "out of", len(listings), "have walking commute times.")

HBox(children=(IntProgress(value=0, max=1106), HTML(value='')))


1043 out of 1106 have park and ride commute times.
780 out of 1106 have walking commute times.


In [8]:
gsmls_filters = lambda x: ((x.address.notnull()) &
                            (~x.city.isin(blacklist_cities)) &
                            (x.barnabas_duration < HALF_HOUR + TEN_MINUTES) &                           
                            ((x.park_and_ride_duration1 < TEN_MINUTES) & (x.park_and_ride_duration < ONE_HOUR_TEN_MINUTES) | (x.nyc_duration < ONE_HOUR_TEN_MINUTES)) &
                            (~x['style'].str.contains('Townhouse', case=False)) &
                            (~x['style'].str.contains('Bi-Level', case=False)) &
                            (~x['style'].str.contains('Cape Cod', case=False)) &
                            (~x['style'].str.contains('Multi Floor Unit', case=False)) &
                            (~x['style'].str.contains('1/2 Duplex', case=False)) &
                            (~x.heat_source.str.contains('oil', case=False, na=True)) &
                            (~x.heat_system.str.contains('radiator', case=False, na=True)) &
                            # (x.cool_system.str.contains('central', case=False, na=True)) &
                            (~x.water.str.contains('well', case=False, na=True)) &
                            (~x.sewer.str.contains('septic', case=False, na=True))
                          )

def show_gsmls():
    df = pd.DataFrame(listings)
    df.set_index('id', inplace=True)
    df = df[gsmls_filters(df)]
    df = df.sort_values(by=['park_and_ride_duration', 'park_and_ride_distance'], ascending=[True, True])
    return df

**GSMLS: Large square footage, low taxes**

In [9]:
df1a = show_gsmls()
df1a = df1a[(df1a.sqft > 2000) & (df1a.tax < 10000)]
df1a = df1a.sort_values('barnabas_duration')
print(len(df1a), "listings after filtering")
preview_gsmls_df(df1a)

8 listings after filtering


,address,city,county,bedrooms,baths_full,baths_part,sqft,price,tax,style,rooms,park_and_ride_name,park_and_ride_duration_text,park_and_ride_type,park_and_ride_distance,park_and_ride_bus_name,park_and_ride_bus_distance,barnabas_duration_text,nyc_duration_text
id,,,,,,,,,,,,,,,,,,,
3424306,980 Moessner Ave,Union,Union,5,3,0,2637.0,429000,8911,Colonial,10,Springfield Center - Springfield Twp,"4 mins, 1 hour 3 mins",bus,0.939318,Springfield Center - Springfield Twp,0.939318,16 mins,57 mins
3435172,6 Boyden Ave,Maplewood,Essex,4,3,1,3724.0,490000,9971,Colonial,9,South Orange,"7 mins, 36 mins",rail,1.054738,Irvington Bus Terminal,1.213625,17 mins,47 mins
3400060,72 Crescent Ave,Totowa,Passaic,3,2,1,2232.0,429000,0,Colonial,8,Little Falls,"6 mins, 1 hour 4 mins",rail,1.310098,Wayne/Route 23 Transit Center,2.166353,30 mins,1 hour 5 mins
3435024,22 Euclid Ave,Ridgefield Park,Bergen,3,2,2,5001.0,479800,8665,Colonial,9,Vince Lombardi,"5 mins, 25 mins",bus,1.637446,Vince Lombardi,1.637446,31 mins,28 mins
3432363,460 Glenwood Ave,Teaneck,Bergen,3,2,0,3419.0,369999,7139,Colonial,6,Hackensack Terminal,"8 mins, 45 mins",bus,2.137270,Hackensack Terminal,2.137270,32 mins,34 mins
3400653,112 S Valley Rd,Lincoln Park,Morris,4,2,1,2371.0,499000,6389,Custom Home,8,Mountain View,"3 mins, 1 hour 11 mins",rail,0.537797,Mothers Park & Ride,0.965760,33 mins,1 hour 3 mins
3434968,6 First St,Pequannock Township,Morris,4,2,1,3300.0,350000,0,Colonial,10,Lincoln Park,"5 mins, 1 hour 5 mins",rail,1.674614,Mothers Park & Ride,2.179765,33 mins,1 hour 3 mins
3424983,24 Yorkshire Pl,Sayreville,Middlesex,6,3,0,7501.0,385000,9034,"Development Home, Ranch",12,Sayreville,"4 mins, 55 mins",bus,0.248533,Sayreville,0.248533,38 mins,1 hour 6 mins


**GSMLS: Large square footage, medium taxes**

In [10]:
df1b = show_gsmls()
df1b = df1b[(df1b.sqft > 2000) & (df1b.tax > 10000) & (df1b.tax < 13500)]
df1b = df1b.sort_values('price')
print(len(df1b), "listings after filtering")
preview_gsmls_df(df1b)

8 listings after filtering


,address,city,county,bedrooms,baths_full,baths_part,sqft,price,tax,style,rooms,park_and_ride_name,park_and_ride_duration_text,park_and_ride_type,park_and_ride_distance,park_and_ride_bus_name,park_and_ride_bus_distance,barnabas_duration_text,nyc_duration_text
id,,,,,,,,,,,,,,,,,,,
3416166,1148-50 CUSHING RD,Plainfield,Union,5,3,0,2016.0,364900,12500,Ranch,13,Netherwood,"4 mins, 1 hour 2 mins",rail,0.825262,Watchung Park & Ride,1.834820,35 mins,1 hour 29 mins
3410954,425 Pine Brook Rd,Lincoln Park,Morris,3,3,1,2406.0,388000,11438,"Colonial, Custom Home",8,Lincoln Park,"3 mins, 1 hour 5 mins",rail,0.693314,Mothers Park & Ride,1.817466,30 mins,1 hour 6 mins
3436476,21 Rockaway Pl,Parsippany-Troy Hills,Morris,4,2,1,2120.0,434900,10208,"Colonial, Split Level",8,Boonton,"10 mins, 1 hour 16 mins",rail,2.532123,Willowbrook Mall,6.263236,21 mins,1 hour 8 mins
3425447,87 Hirliman Rd,Teaneck,Bergen,4,2,1,2560.0,475000,11446,Split Level,10,New Bridge Landing,"8 mins, 49 mins",rail,2.167687,Hackensack Terminal,2.618347,40 mins,38 mins
3433211,126 Linwood Ter,Clifton,Passaic,3,3,0,2200.0,489500,11067,Colonial,7,Allwood Road,"5 mins, 33 mins",bus,0.897368,Allwood Road,0.897368,27 mins,41 mins
3378717,4514 Liberty Ave,North Bergen,Hudson,4,2,1,2913.0,499000,10595,Colonial,9,Tonnelle Avenue,"1 min, 26 mins",light_rail,0.232849,North Bergen,0.742872,31 mins,30 mins
3434419,157 Indian Run Pkwy,Union,Union,4,4,0,2800.0,499000,10477,"Colonial, Custom Home",11,Union Center - Union Twp,"4 mins, 47 mins",bus,0.639539,Union Center - Union Twp,0.639539,23 mins,1 hour 9 mins
3430057,7 Robinwood Drive,Little Falls,Passaic,5,3,1,3080.0,499900,12758,Colonial,9,Montclair State University,"6 mins, 57 mins",rail,1.020244,Allwood Road,2.453861,25 mins,50 mins


**GSMLS: Large square footage, high taxes**

In [11]:
df1bb = show_gsmls()
df1bb = df1bb[(df1bb.sqft > 2000) & (df1bb.tax > 13500) & (df1bb.tax < 16000)]
df1bb = df1bb.sort_values('tax')
print(len(df1bb), "listings after filtering")
preview_gsmls_df(df1bb)

6 listings after filtering


,address,city,county,bedrooms,baths_full,baths_part,sqft,price,tax,style,rooms,park_and_ride_name,park_and_ride_duration_text,park_and_ride_type,park_and_ride_distance,park_and_ride_bus_name,park_and_ride_bus_distance,barnabas_duration_text,nyc_duration_text
id,,,,,,,,,,,,,,,,,,,
3434778,79 Winding Ln,Bloomfield,Essex,4,3,0,3054.0,489000,13683,Split Level,11,Watchung Avenue,"6 mins, 45 mins",rail,1.334357,Allwood Road,1.641766,24 mins,56 mins
3433347,340 Walker Rd,West Orange,Essex,3,2,1,2066.0,440000,13755,Split Level,9,Mountain Station,"5 mins, 43 mins",rail,1.151145,Irvington Bus Terminal,3.526863,8 mins,1 hour 37 mins
3436125,12 Ridge Ave,Little Falls,Passaic,4,3,1,3170.0,439000,13938,Colonial,10,Little Falls,"3 mins, 1 hour 4 mins",rail,0.408777,Willowbrook Mall,1.827223,24 mins,50 mins
3418703,35 Van Winkle Ct,Woodland Park,Passaic,3,3,0,2500.0,499900,14610,Ranch,7,Montclair State University,"5 mins, 57 mins",rail,1.174653,Allwood Road,2.679599,30 mins,1 hour 5 mins
3420007,1480 Ratzer Rd,Wayne,Passaic,6,4,0,3490.0,459900,14789,"See Remarks, Split Level",10,Broadway Bus Terminal - Paterson,"12 mins, 59 mins",bus,2.841251,Broadway Bus Terminal - Paterson,2.841251,36 mins,1 hour 5 mins
3411678,47 NORMANDY RD,Woodbridge Township,Middlesex,5,3,1,3705.0,499900,15550,Colonial,9,Metropark,"10 mins, 38 mins",rail,2.482587,Watchung Park & Ride,4.909467,36 mins,1 hour 34 mins


**GSMLS: Null square footage, low taxes**

In [12]:
df1c = show_gsmls()
df1c = df1c[(df1c.sqft.isnull()) & (df1c.tax < 10000)]
df1c = df1c.sort_values('city')
print(len(df1c), "listings after filtering")
preview_gsmls_df(df1c)

60 listings after filtering


,address,city,county,bedrooms,baths_full,baths_part,sqft,price,tax,style,rooms,park_and_ride_name,park_and_ride_duration_text,park_and_ride_type,park_and_ride_distance,park_and_ride_bus_name,park_and_ride_bus_distance,barnabas_duration_text,nyc_duration_text
id,,,,,,,,,,,,,,,,,,,
3428699,27 Ira Rd,Cedar Grove,Essex,4,2,0,NaN,438880,7619,Colonial,10,Upper Montclair,"6 mins, 48 mins",rail,1.039111,Allwood Road,2.897116,20 mins,51 mins
3433324,217 Stevens Ave,Cedar Grove,Essex,3,3,1,NaN,509000,9231,Colonial,6,Little Falls,"2 mins, 1 hour 4 mins",rail,0.423710,Willowbrook Mall,1.753137,23 mins,1 hour 4 mins
3428633,447 Westfield Ave,Clark,Union,4,2,1,NaN,399999,9631,"Custom Home, Split Level",10,Garwood,"9 mins, 53 mins",rail,1.974796,Rutgers Lane Hospital - Union Twp,4.251719,29 mins,1 hour 30 mins
3436414,44 KATHRYN ST,Clark,Union,4,2,0,NaN,469900,8551,"Custom Home, See Remarks",7,Rahway,"7 mins, 48 mins",rail,1.507395,Carteret,4.256120,30 mins,1 hour 23 mins
3428897,80 Fulton St,Clark,Union,3,2,1,NaN,449900,8584,Ranch,8,Rahway,"7 mins, 48 mins",rail,1.516515,Rutgers Lane Hospital - Union Twp,3.911853,29 mins,1 hour 21 mins
3435731,42 Livingston St,Clifton,Passaic,3,2,1,NaN,478888,7605,Colonial,9,Clifton,"7 mins, 44 mins",rail,1.562199,Allwood Road,2.143985,27 mins,55 mins
3435342,52 Rowland Ave,Clifton,Passaic,4,2,1,NaN,399000,8047,Custom Home,8,Passaic,"4 mins, 40 mins",rail,0.768880,Allwood Road,1.420538,30 mins,48 mins
3422738,119 Edgewood Ave,Clifton,Passaic,3,2,1,NaN,374900,9515,Colonial,6,Passaic,"4 mins, 40 mins",rail,0.830379,Clifton Commons,1.141417,29 mins,51 mins
3425188,21 Clay St,Clifton,Passaic,4,2,1,NaN,389900,8905,See Remarks,7,Clifton Commons,"3 mins, 47 mins",bus,0.283583,Clifton Commons,0.283583,28 mins,46 mins


**GSMLS: Null square footage, medium taxes**

In [13]:
df1d = show_gsmls()
df1d = df1d[(df1d.sqft.isnull()) & (df1d.tax > 10000) & (df1d.tax < 13500)]
df1d = df1d.sort_values('city')
print(len(df1d), "listings after filtering")
preview_gsmls_df(df1d)

51 listings after filtering


,address,city,county,bedrooms,baths_full,baths_part,sqft,price,tax,style,rooms,park_and_ride_name,park_and_ride_duration_text,park_and_ride_type,park_and_ride_distance,park_and_ride_bus_name,park_and_ride_bus_distance,barnabas_duration_text,nyc_duration_text
id,,,,,,,,,,,,,,,,,,,
3425147,85 Bellevue Ter,Bloomfield,Essex,3,2,1,NaN,424900,12668,Split Level,8,Allwood Road,"4 mins, 33 mins",bus,1.063173,Allwood Road,1.063173,27 mins,58 mins
3400950,47 Hearthstone Rd,Bloomfield,Essex,4,2,0,NaN,350000,12699,Split Level,10,Watchung Avenue,"6 mins, 45 mins",rail,1.459975,Allwood Road,1.638773,25 mins,57 mins
3430823,6 Colony Dr,Caldwell,Essex,4,2,0,NaN,430000,12856,Colonial,7,Little Falls,"6 mins, 1 hour 4 mins",rail,1.248499,Willowbrook Mall,1.470499,20 mins,1 hour 15 mins
3428051,42 KENNETH PL,Clark,Union,3,2,0,NaN,439000,10643,Split Level,8,Garwood,"10 mins, 53 mins",rail,2.017407,Rutgers Lane Hospital - Union Twp,4.342921,29 mins,1 hour 32 mins
3435953,48 Godwin Pl,Clifton,Passaic,3,2,1,NaN,399000,13426,Split Level,8,Allwood Road,"3 mins, 33 mins",bus,0.553667,Allwood Road,0.553667,27 mins,53 mins
3433595,1 Friar Ln,Clifton,Passaic,3,2,1,NaN,489000,11135,Custom Home,7,Allwood Road,"4 mins, 33 mins",bus,0.846560,Allwood Road,0.846560,25 mins,44 mins
3401881,41 Ridgewood Rd,Clifton,Passaic,4,3,0,NaN,399900,11114,Colonial,9,Allwood Road,"4 mins, 33 mins",bus,0.702121,Allwood Road,0.702121,28 mins,39 mins
3432605,790 Grove St,Clifton,Passaic,3,2,0,NaN,499000,12971,Ranch,6,Montclair Heights,"4 mins, 52 mins",rail,0.712505,Allwood Road,0.777666,25 mins,52 mins
3414582,114 Allwood Pl,Clifton,Passaic,4,2,0,NaN,499000,11345,"Custom Home, Ranch",8,Passaic,"4 mins, 40 mins",rail,0.769685,Clifton Commons,1.376629,30 mins,51 mins


**GSMLS: Null square footage, high taxes**

In [14]:
df1dd = show_gsmls()
df1dd = df1dd[(df1dd.sqft.isnull()) & (df1dd.tax > 13500) & (df1dd.tax < 16000)]
df1dd = df1dd.sort_values('city')
print(len(df1dd), "listings after filtering")
preview_gsmls_df(df1dd)

11 listings after filtering


,address,city,county,bedrooms,baths_full,baths_part,sqft,price,tax,style,rooms,park_and_ride_name,park_and_ride_duration_text,park_and_ride_type,park_and_ride_distance,park_and_ride_bus_name,park_and_ride_bus_distance,barnabas_duration_text,nyc_duration_text
id,,,,,,,,,,,,,,,,,,,
3419648,13 BELLEVUE AVE,Bloomfield,Essex,3,3,0,NaN,425000,14104,Colonial,7,Watchung Avenue,"5 mins, 45 mins",rail,1.211963,Allwood Road,1.266570,26 mins,52 mins
3417461,15 George St,Bloomfield,Essex,4,2,1,NaN,439000,14719,Colonial,8,Watchung Avenue,"6 mins, 45 mins",rail,1.388071,Allwood Road,1.715933,22 mins,48 mins
3424669,3 Trella Ter,Clifton,Passaic,4,3,0,NaN,499000,13539,"Custom Home, Ranch, Raised Ranch",10,Clifton,"8 mins, 44 mins",rail,1.260493,Allwood Road,1.546734,28 mins,47 mins
3429553,9 Kennedy Ct,Clifton,Passaic,3,2,1,NaN,443000,15716,Split Level,8,Montclair Heights,"4 mins, 52 mins",rail,0.601613,Allwood Road,0.857479,27 mins,48 mins
3431784,66 Ridge Rd,Hawthorne,Passaic,3,2,1,NaN,499000,15311,Split Level,10,Glen Rock Main Line,"7 mins, 48 mins",rail,1.533489,Midland Park,2.290814,39 mins,1 hour 4 mins
3419601,3 W Walnut St,Metuchen,Middlesex,5,2,1,NaN,487000,13848,Colonial,7,Metuchen,"3 mins, 51 mins",rail,0.560384,Sayreville,4.991073,38 mins,1 hour 11 mins
3421163,201 Maple Ave,Metuchen,Middlesex,5,2,2,NaN,500000,14096,Ranch,13,Metuchen,"3 mins, 51 mins",rail,0.447927,Sayreville,5.845925,38 mins,1 hour 16 mins
3424594,1470 Cooper Rd,Scotch Plains,Union,4,2,0,NaN,499000,15959,Expanded Ranch,8,Fanwood,"6 mins, 58 mins",rail,1.473570,Watchung Park & Ride,2.527674,34 mins,1 hour 54 mins
3424889,1561 Cooper Rd,Scotch Plains,Union,4,2,0,NaN,449900,14690,Colonial,9,Fanwood,"6 mins, 58 mins",rail,1.365185,Watchung Park & Ride,2.500140,34 mins,1 hour 35 mins
